# Coffea-Casa Processor-Based Workflow Test

This notebook demonstrates the UnifiedProcessor workflow with coffea.processor.Runner on Coffea-Casa, including skimming, analysis, histogramming, and statistics steps.

## Workflow Overview

1. Setup Python path for intccms package
2. Install dependencies and register modules for cloud pickle
3. Acquire Dask client from Coffea-Casa environment
4. Configure analysis parameters
5. Run metadata extraction
6. Initialize UnifiedProcessor
7. Run processor with coffea.processor.Runner
8. Save histograms
9. Run statistical analysis (if enabled)

In [1]:
# Setup Python path to include intccms package
import sys
import time
from pathlib import Path

# Add src directory to Python path
repo_root = Path.cwd()
src_dir = repo_root / "src"
examples_dir = repo_root
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))
if str(examples_dir) not in sys.path:
    sys.path.insert(0, str(examples_dir))
print(f"✅ Added {src_dir} to Python path")
print(f"✅ Added {examples_dir} to Python path")

✅ Added /home/cms-jovyan/integration-challenge/cms/src to Python path
✅ Added /home/cms-jovyan/integration-challenge/cms to Python path


In [2]:
COFFEA_VERSION = "=2025.11.0"
COFFEA_PIP = f"coffea=={COFFEA_VERSION}"
DEPS = [COFFEA_PIP]
try:
    import omegaconf
except ImportError:
    print("⚠️ omegaconf not found, installing...")
    ! pip install omegaconf

In [3]:
# Imports and cloudpickle registration
import copy
import os

os.environ['AWS_ACCESS_KEY_ID'] = ""
os.environ['AWS_SECRET_ACCESS_KEY'] = ""

from dask.distributed import Client, PipInstall
from coffea.processor import DaskExecutor, IterativeExecutor
from coffea.nanoevents import NanoAODSchema

import cloudpickle
import intccms
import example_cms

# Register modules for cloud pickle
cloudpickle.register_pickle_by_value(intccms)
cloudpickle.register_pickle_by_value(example_cms)

from example_cms.configs.configuration import config as original_config
from intccms.schema import Config, load_config_with_restricted_cli
from intccms.utils.output import OutputDirectoryManager
from intccms.metadata_extractor import DatasetMetadataManager
from intccms.datasets import DatasetManager
from intccms.analysis import run_processor_workflow

## Acquire Dask Client

Coffea-Casa provides a shared scheduler. Connect to it and register dependencies.

In [9]:
from dask.distributed import WorkerPlugin
from contextlib import contextmanager

class RedirectStderrToStdout(WorkerPlugin):
    def setup(self, worker):
        # crude but effective: route stderr to stdout
        sys.stderr = sys.stdout

@contextmanager
def acquire_client(af="gateway"):
    """Context manager to acquire and safely close a Dask client from a Coffea-Casa environment."""
    client = None
    cluster = None
    try:
        if af == "condor":
            client = Client("tls://localhost:8786")
            client.register_plugin(PipInstall(packages=DEPS))
            cluster = None
        
        elif af == "gateway":
            def set_env(dask_worker):
                config_path = str(Path(dask_worker.local_directory) / 'access_token')
                os.environ["BEARER_TOKEN_FILE"] = config_path
                os.chmod(config_path, 0o600)
                os.chmod("/etc/grid-security/certificates", 0o755)
    
            num_workers = 350   #number of workers desired
            from dask_gateway import Gateway
            gateway = Gateway()
            clusters = gateway.list_clusters()
            cluster = gateway.connect(clusters[0].name)
            client = cluster.get_client()
            cluster.scale(num_workers)
            client.wait_for_workers(num_workers)
            client.upload_file("/etc/cmsaf-secrets-chown/access_token")     
            client.register_worker_callbacks(setup=set_env)
            client.register_plugin(PipInstall(packages=DEPS))
            # Register from client *before* workers start
            client.register_plugin(RedirectStderrToStdout(), name="redirect-stderr")
        
        print(f"✅ Connected to Dask scheduler")
        print(f"📊 Dashboard: {client.dashboard_link}")
        
        yield client, cluster
    finally:
        if client is not None:
            client.close()

## Configuration Setup

Configure analysis parameters including which processes to run and output settings.

In [10]:
# Configuration setup
config = copy.deepcopy(original_config)

# Limit files for testing
config["datasets"]["max_files"] = None

# Use local output directory
config["general"]["output_dir"] = "example_cms/outputs/"

# Configuration flags
config["general"]["read_from_cache"] = False
config["general"]["run_metadata_generation"] = True
config["general"]["run_processor"] = True  # Set to False to skip processor and load saved histograms
config["general"]["save_skimmed_output"] = False  # Set to True to save filtered events to disk
config["general"]["run_analysis"] = True
config["general"]["run_histogramming"] = True
config["general"]["run_systematics"] = True
config["general"]["run_statistics"] = False

# Test only signal dataset
#config["general"]["processes"] = ["data"]

cli_args = []
full_config = load_config_with_restricted_cli(config, cli_args)
validated_config = Config(**full_config)

## Run Complete Workflow

Execute the full processor workflow with proper cleanup in a try/finally block.

In [11]:
# Set up output manager
output_manager = OutputDirectoryManager(
    root_output_dir=validated_config.general.output_dir,
    cache_dir=validated_config.general.cache_dir,
    metadata_dir=validated_config.general.metadata_dir,
    skimmed_dir=validated_config.general.skimmed_dir
)

16:10:18 INFO     Output directory manager initialized with root:                                ]8;id=310056;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/utils/output/directories.py\directories.py]8;;\:]8;id=892541;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/utils/output/directories.py#169\169]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example_cms/outputs                                   

In [12]:
# Set up metadata generator from datasets
# TODO:: update method for redirector?
dataset_manager = DatasetManager(validated_config.datasets)

         INFO     Initialized dataset manager with 10 datasets                                        ]8;id=437327;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/datasets/manager.py\manager.py]8;;\:]8;id=858713;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/datasets/manager.py#34\34]8;;\

In [13]:
# Extract the metadata
with acquire_client(af="gateway") as (client, cluster):
    metadata_generator = DatasetMetadataManager(
        dataset_manager=dataset_manager,
        output_manager=output_manager,
        executor=DaskExecutor(client=client),
        config=validated_config,
    )
    metadata_generator.run(
        generate_metadata=validated_config.general.run_metadata_generation,
        processes_filter=validated_config.general.processes if hasattr(validated_config.general, 'processes') else None
    )

✅ Connected to Dask scheduler
📊 Dashboard: /services/dask-gateway/clusters/cmsaf-dev.70b76ebddab44bd8890f0ab9631b10ee/status


16:10:23 INFO     Initialized DatasetMetadataManager with output dir:                                ]8;id=241571;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=179794;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#131\131]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example_cms/outputs/metadata                          

         INFO     Starting metadata generation workflow...                                           ]8;id=31244;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=48711;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#194\194]8;;\

         INFO     Building fileset for process: signal                                              ]8;id=158993;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/builders.py\builders.py]8;;\:]8;id=984147;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/builders.py#123\123]8;;\

         INFO     Building fileset for process: ttbar_semilep                                       ]8;id=167712;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/builders.py\builders.py]8;;\:]8;id=758246;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/builders.py#123\123]8;;\

...

         INFO     Saved JSON to                                                                           ]8;id=118835;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/io.py\io.py]8;;\:]8;id=583700;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/io.py#107\107]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example_cms/outputs/metadata/nanoaods_data_1          
                  0_nominal.json                                                                                   

         INFO     Saved JSON to                                                                           ]8;id=557065;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/io.py\io.py]8;;\:]8;id=291001;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/io.py#107\107]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example_cms/outputs/metadata/nanoaods_data_1          
                  1_nominal.json                                                                                   

         INFO     Saved JSON to                                                                           ]8;id=511550;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/io.py\io.py]8;;\:]8;id=616463;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/io.py#107\107]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example_cms/outputs/metadata/nanoaods_data_1          
                  2_nominal.json                                                                                   

         INFO     Saved JSON to                                                                           ]8;id=730594;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/io.py\io.py]8;;\:]8;id=843536;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/io.py#107\107]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example_cms/outputs/metadata/nanoaods_data_1          
                  3_nominal.json                                                                                   

         INFO     Metadata generation complete.                                                      ]8;id=574013;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=177017;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#210\210]8;;\

In [14]:
# Build look up table for metadata and extract workitems
metadata_lookup = metadata_generator.build_metadata_lookup()
workitems = metadata_generator.workitems
print(f"✅ Generated {len(workitems)} workitems")

16:12:59 INFO     Built metadata lookup for 125 fileset keys                                         ]8;id=445375;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/manager.py\manager.py]8;;\:]8;id=998678;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/metadata_extractor/manager.py#427\427]8;;\

✅ Generated 39603 workitems


In [15]:
# Run processor workflow 
with acquire_client(af="gateway") as (client, cluster):
    print("\n🚀 Running processor workflow...")
    t0 = time.perf_counter()
    output, report, metrics = run_processor_workflow(
        config=validated_config,
        output_manager=output_manager,
        metadata_lookup=metadata_lookup,
        workitems=workitems,
        executor=DaskExecutor(client=client, treereduction=8, retries=0),
        schema=NanoAODSchema,
    )
    t1 = time.perf_counter()
    print(f"✅ Processor workflow complete in {t1-t0} seconds!")

# Print a small summary of number of events processed
if validated_config.general.run_processor:
    print(f"📊 Total events processed: {output.get('processed_events', 0):,}")
    if 'skimmed_events' in output:
        print(f"✂️  Events after skim: {output.get('skimmed_events', 0):,}")

✅ Connected to Dask scheduler
📊 Dashboard: /services/dask-gateway/clusters/cmsaf-dev.70b76ebddab44bd8890f0ab9631b10ee/status

🚀 Running processor workflow...


16:15:13 INFO     Running processor over data...                                                      ]8;id=270301;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=77532;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py#125\125]8;;\

         INFO     Initialized UnifiedProcessor: save_skimmed_output=False, analysis=True,          ]8;id=831770;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/processor.py\processor.py]8;;\:]8;id=461153;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/processor.py#194\194]8;;\
                  histogramming=True, systematics=True                                                             

         INFO     Started worker tracking on scheduler                                                ]8;id=950092;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=645894;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py#216\216]8;;\

         INFO     Processing 39603 work items with chunksize=200000                                   ]8;id=47494;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=887910;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py#247\247]8;;\

...

16:19:29 INFO     Saved worker timeline to                                                            ]8;id=829382;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=981454;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py#278\278]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example_cms/outputs/benchmarks/20251209-              
                  161513                                                                                           

         INFO     Saved metrics measurement to                                                        ]8;id=793899;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=621202;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py#296\296]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example_cms/outputs/benchmarks/20251209-              
                  161513                                                                                           

         INFO     📊 Dask performance report:                                                         ]8;id=425377;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=116051;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py#300\300]8;;\
                  /home/cms-jovyan/integration-challenge/cms/example_cms/outputs/benchmarks/20251209-              
                  161513/dask_performance.html                                                                     

         INFO     📈 Metrics: 19.71 Gbps, 26788.0 kHz                                                 ]8;id=503659;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py\runner.py]8;;\:]8;id=365248;file:///home/cms-jovyan/integration-challenge/cms/src/intccms/analysis/runner.py#303\303]8;;\

✅ Processor workflow complete in 256.25230254698545 seconds!
📊 Total events processed: 6,827,277,103
✂️  Events after skim: 297,458,382


In [ ]:
print(f"data read: {report["bytesread"] / 1000**3:.2f} GB in {report["chunks"]} chunks")

print(f"core-average event rate using \'processtime\': {report["entries"] / 1000 / report["processtime"]:.2f} kHz")
print(f"core-average data rate using \'processtime\': {report["bytesread"] / 1000**3 * 8 / report["processtime"]:.2f} Gbps")

print(f"average event rate using walltime: {report["entries"] / 1000 / (t1 - t0):.2f} kHz")
print(f"average data rate using walltime: {report["bytesread"] / 1000**3 * 8 / (t1 - t0):.2f} Gbps")

print(f"Number of branches read: {len(report["columns"])}")

In [ ]:
# Import Rich for beautiful table display
from rich.console import Console

console = Console()

print("\n" + "=" * 60)
print("📊 Processing Metrics")
print("=" * 60)

In [ ]:
# Display metrics if collection was enabled
if metrics:
    from intccms.metrics import (
        format_throughput_table,
        format_event_processing_table,
        format_resources_table,
        format_timing_table,
    )
    
    print("\n📈 Throughput Metrics")
    console.print(format_throughput_table(metrics))
    
    print("\n⚡ Event Processing Metrics")
    console.print(format_event_processing_table(metrics))
    
    print("\n🖥️  Resource Utilization")
    console.print(format_resources_table(metrics))
    
    print("\n⏱️  Timing Breakdown")
    console.print(format_timing_table(metrics))
else:
    print("⚠️  Metrics collection was disabled (set config.general.metrics.enable=True)")

In [ ]:
# Step 4: Run Statistical Analysis
if validated_config.general.run_statistics and output and "histograms" in output:
    print("\n📊 Running statistical analysis...")

    # Create analysis instance for statistics
    from intccms.analysis.nondiff import NonDiffAnalysis

    analysis = NonDiffAnalysis(validated_config, output_manager)
    # Set histograms from processor output
    analysis.nD_hists_per_region = output["histograms"]

    # Check if cabinetry config exists
    if hasattr(validated_config, 'statistics') and hasattr(validated_config.statistics, 'cabinetry_config'):
        cabinetry_config_path = validated_config.statistics.cabinetry_config

        # Check if file exists
        if Path(cabinetry_config_path).exists():
            print(f"✅ Using cabinetry config: {cabinetry_config_path}")
            analysis.run_statistics(cabinetry_config_path)
            print(f"✅ Statistical analysis complete!")
            print(f"📊 Plots saved to: {output_manager.statistics_dir}")
        else:
            print(f"⚠️  Cabinetry config not found: {cabinetry_config_path}")
            print(f"   Skipping statistics step")
    else:
        print(f"⚠️  No cabinetry_config specified in configuration")
        print(f"   Skipping statistics step")
else:
    print("\n⚠️  Statistics step skipped (disabled or no histograms)")
